In [2]:
import pandas as pd

### Drugs to medical conditions

In [12]:
# from https://www.kaggle.com/datasets/jithinanievarghese/drugs-related-to-common-treatments?select=drugs_for_common_treatments.csv
df_drugs_conditions = pd.read_csv('drugs_for_common_treatments.csv') 
df_drugs_conditions = df_drugs_conditions.iloc[:, :2]
df_drugs_conditions.iloc[:, 0] = df_drugs_conditions.iloc[:, 0].str.lower()
df_drugs_conditions.iloc[:, 1] = df_drugs_conditions.iloc[:, 1].str.lower()
df_drugs_conditions.to_csv('../clean_datasets/drugs_to_medical_conditions.csv', index=False)

df_drugs_conditions



,drug_name,medical_condition
0,doxycycline,acne
1,spironolactone,acne
2,minocycline,acne
3,accutane,acne
4,clindamycin,acne
...,...,...
3954,fenfluramine,weight loss
3955,fendique er,weight loss
3956,imcivree,weight loss
3957,setmelanotide,weight loss


### Symptoms to medical conditions

In [9]:
df_symptoms_conditions = pd.read_csv('https://raw.githubusercontent.com/anujdutt9/Disease-Prediction-from-Symptoms/refs/heads/master/dataset/training_data.csv')
df_symptoms_conditions = df_symptoms_conditions.iloc[:, :133]

def create_symptoms_by_prognosis_csv(df, output_filename='../clean_datasets/symptoms_by_prognosis.csv'):
    """
    Process a DataFrame containing symptom data (1s and 0s) and prognosis
    to create a CSV maintaining each unique symptom combination for each prognosis.
    
    Args:
        df (pandas.DataFrame): DataFrame with symptoms as columns and prognosis in last column
        output_filename (str): Name of the output CSV file
    """
    # Get all column names except the last one (prognosis)
    symptom_columns = df.columns[:-1]
    
    # List to store each row of output data
    output_data = []
    
    # Process each row in the original DataFrame
    for idx, row in df.iterrows():
        prognosis = row.iloc[-1]  # Get the prognosis (last column)
        
        # Find symptoms that are present (value == 1)
        present_symptoms = []
        for symptom in symptom_columns:
            if row[symptom] == 1:
                present_symptoms.append(symptom)
        
        # Add to output data
        output_data.append({
            'Prognosis': prognosis,
            'Symptoms': ', '.join(present_symptoms)
        })
    
    # Convert to DataFrame and save to CSV
    output_df = pd.DataFrame(output_data)
    output_df.to_csv(output_filename, index=False)
    print(f"CSV file '{output_filename}' has been created successfully!")
    
    # Display some statistics
    print(f"\nTotal number of symptom combinations: {len(output_df)}")
    print(f"Number of unique prognoses: {output_df['Prognosis'].nunique()}")
    print("\nSample of first few rows:")
    print(output_df.head())
    
    return output_df

create_symptoms_by_prognosis_csv(df_symptoms_conditions)

CSV file '../clean_datasets/symptoms_by_prognosis.csv' has been created successfully!

Total number of symptom combinations: 4920
Number of unique prognoses: 41

Sample of first few rows:
          Prognosis                                           Symptoms
0  Fungal infection  itching, skin_rash, nodal_skin_eruptions, disc...
1  Fungal infection  skin_rash, nodal_skin_eruptions, dischromic _p...
2  Fungal infection  itching, nodal_skin_eruptions, dischromic _pat...
3  Fungal infection            itching, skin_rash, dischromic _patches
4  Fungal infection           itching, skin_rash, nodal_skin_eruptions


,Prognosis,Symptoms
0,Fungal infection,"itching, skin_rash, nodal_skin_eruptions, disc..."
1,Fungal infection,"skin_rash, nodal_skin_eruptions, dischromic _p..."
2,Fungal infection,"itching, nodal_skin_eruptions, dischromic _pat..."
3,Fungal infection,"itching, skin_rash, dischromic _patches"
4,Fungal infection,"itching, skin_rash, nodal_skin_eruptions"
...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,"vomiting, headache, nausea, spinning_movements..."
4916,Acne,"skin_rash, pus_filled_pimples, blackheads, scu..."
4917,Urinary tract infection,"burning_micturition, bladder_discomfort, foul_..."
4918,Psoriasis,"skin_rash, joint_pain, skin_peeling, silver_li..."


### .csv to SQL

In [13]:
sql_patient_info = pd.read_csv('../clean_datasets/patient_info.csv')
sql_patient_info = sql_patient_info.rename(columns={
    'First Name': 'FirstName',
    'Last Name': 'LastName'
})
sql_patient_info['Gender'] = sql_patient_info['Gender'].replace({
    'Male': 'm',
    'Female': 'f'
})
sql_patient_info = sql_patient_info[['Username', 'FirstName', 'LastName', 'Gender', 'Age']]
sql_patient_info.to_csv('../sql_patient.csv')


In [9]:
sql_medication = pd.read_csv('../clean_datasets/drugs_to_medical_conditions.csv')
sql_medication = sql_medication.rename(columns={'drug_name': 'MedicationName'})
sql_medication['Dosage'] = ''
sql_medication['TimeIntervals'] = ''
sql_medication = sql_medication[['MedicationName', 'Dosage', 'TimeIntervals']]
sql_medication.to_csv('../sql_medication.csv')


In [10]:
df = pd.read_csv('../clean_datasets/drugs_to_medical_conditions.csv', names=['MedicationName', 'MedicalCondition'])

unique_conditions = df['MedicalCondition'].unique()
condition_to_group = {condition: idx + 1 for idx, condition in enumerate(unique_conditions)}

df['TreatmentGroupID'] = df['MedicalCondition'].map(condition_to_group)

df = df[['TreatmentGroupID', 'MedicationName', 'MedicalCondition']]
df.to_csv('../sql_possible_treatments.csv')

In [ ]:
df1 = pd.read_csv('../clean_datasets/symptoms_by_prognosis.csv')
df1 = df1.rename(columns={'Prognosis': 'MedicalCondition'})
df1.to_csv('../sql_diagnosis.csv')